In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

In [33]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy không hiển thị trình duyệt
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

In [34]:
driver = webdriver.Chrome()  # Không cần chỉ định ChromeDriver
driver.get("https://www.google.com")

In [35]:
driver.get("https://www.livecareer.com/resume-examples")
time.sleep(5)
print(driver.page_source)

<html lang="en-US"><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta http-equiv="X-UA-Compatible" content="ie=edge"> <script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-971063891&amp;l=dataLayer&amp;cx=c&amp;gtm=45je53j1v871411692za200zb77769872&amp;tag_exp=102482433~102788824~102803279~102813109"></script><script src="//bat.bing.com/bat.js" async=""></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-TEPPBW8QXP&amp;l=dataLayer&amp;cx=c&amp;gtm=45He53j1v77769872za200&amp;tag_exp=102482433~102788824~102803279~102813109~102887800"></script><script type="text/javascript" src="https://cse.expertrec.com/js/ci_customSearch_separate.1742557672350.min.js"></script><script type="text/javascript" defer="" src="https://cdn.segment.com/analytics.js/v1/B7rLk975WsvRuE8E91XEVqzL3jn8HddD/analytics.min.js"></script><script type="text/javascript" src="https

In [36]:
# Chờ tải trang đầy đủ
time.sleep(5)

In [37]:
# Cuộn xuống để tải toàn bộ nội dung (Lazy Load)
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [38]:
# Tìm tất cả các phần tử chứa thông tin CV
cv_elements = driver.find_elements(By.CLASS_NAME, "mini-resume")

data = []
for cv in cv_elements:
    try:
        # Lấy danh mục (Category)
        category = cv.find_element(By.CLASS_NAME, "resume-job-title").text.strip()

        # Lấy thẻ <img>
        img_element = cv.find_element(By.CSS_SELECTOR, "img.lazy-hidden")

        # Kiểm tra xem có thuộc tính data-src hoặc data-lazy-src không
        img_link = img_element.get_attribute("data-src") or img_element.get_attribute("data-lazy-src")

        # Nếu không có data-src hoặc data-lazy-src, thử lấy src và đợi nó thay đổi
        if not img_link:
            # Đợi cho đến khi src thay đổi từ placeholder
            WebDriverWait(driver, 10).until(lambda d: "data:image" not in img_element.get_attribute("src"))
            img_link = img_element.get_attribute("src")

        # Kiểm tra xem img_link có hợp lệ không
        if not img_link or "data:image" in img_link:
            print(f"⚠ {category} - Không tìm thấy link ảnh hợp lệ")
            continue

        # Lưu vào danh sách
        data.append([category, img_link])

        print(f"✔ {category} - {img_link}")

    except Exception as e:
        print(f"❌ Lỗi khi lấy dữ liệu: {e}")

✔ Accounting Manager - https://www.livecareer.com/lcapp/uploads/2024/12/accounting-manager-example-resume.svg
✔ Bookkeeper - https://www.livecareer.com/lcapp/uploads/2022/12/bookkeeper-example-resume.svg
✔ Billing Specialist - https://www.livecareer.com/lcapp/uploads/2021/11/billing-specialist-resume-example.svg
✔ Operations Manager - https://www.livecareer.com/lcapp/uploads/2024/12/operations-manager-example-resume.svg
✔ Babysitter - https://www.livecareer.com/lcapp/uploads/2023/10/babysitter-example-resume.svg
✔ Java Developer - https://www.livecareer.com/lcapp/uploads/2022/04/java-developer-esteemed-resume-example-chronological-1.svg
✔ Project Manager - https://www.livecareer.com/lcapp/uploads/2022/12/construction-project-manager-example-resume.svg
✔ Salon Manager - https://www.livecareer.com/lcapp/uploads/2024/12/salon-manager-example-resume.svg
✔ Prep Cook Assistant - https://www.livecareer.com/lcapp/uploads/2022/12/prep-cook-example-resume.svg
✔ Customer Service Representaitve - 

In [39]:
# Đóng trình duyệt
driver.quit()

In [42]:
import csv
# Lưu vào CSV
csv_file = "D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv_links.csv"
with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Category", "Resume_Link"])
    writer.writerows(data)

print(f"✅ Đã lưu dữ liệu vào {csv_file}")

✅ Đã lưu dữ liệu vào D:/BaiDoAnChuyenNganh3/Automated-Resume-Ranking-System-main/csvfiles/cv_links.csv
